In [1]:
# prep for tts segment analysis 
# want to isolate each awas cannister in the UT and get TOGA measurements within time span
# date created: 2/28/2020
# author: sofia chelpon 

In [7]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns

import datetime as dt 

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [8]:
#### ------------------- TIME SPAN 
# 6 minutes total, 3 mins of either side of awas collection 
m = 6
speed = 235.4 #m/s
dist_km = (speed*(m*60))/1000
dist_km

84.744

In [9]:
#### ------------------- LOAD IN 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_twp_noRF06.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df_twp_noRF06.pkl")

In [10]:
#### ------------------- ISOLATE AWAS SEGMENTS IN THE UT 
awas_df_ut = awas_df[(awas_df['GGALT'] > 12000) & (awas_df['GGALT'] < 14000)]
awas_df_ut.reset_index()

,index,Time_UTC,Flight,Notes,Intrument,GGALT,GGLAT,GGLON,C2H6_Ethane,C2H2_Ethyne,...,C2Br2F4_Halon_2402,CH2BrCl_Bromochloromethane,CH2Br2_Methylene_bromide,CHBrCl2_Bromodichloromethane,CHBr2Cl_Dibromochloromethane,CHBr3_Bromoform,C2H5ONO2_Ethyl_nitrate,C3H7NO3_Isopropyl_nitrate,C4H9NO3_2_Butyl_nitrate,C4H9NO3_n_Butyl_nitrate
0,13,2014-01-22 01:44:00,RF05,"Convective outflow, 2 - North of Guam",AWAS,12925.989258,12.914162,149.354141,418.0,38.0,...,0.43,0.14,0.99,0.48,0.25,0.74,3.23,1.62,0.67,0.00
1,14,2014-01-22 01:55:15,RF05,"Convective outflow, 2 - North of Guam",AWAS,12923.607422,14.304468,149.771027,422.0,42.0,...,0.41,0.13,0.98,NaN,0.23,0.70,3.25,1.52,0.78,0.00
2,15,2014-01-22 02:06:30,RF05,"Convective outflow, 2 - North of Guam",AWAS,12921.866211,15.739713,150.000473,360.0,34.0,...,0.41,0.12,0.96,NaN,0.21,0.67,3.07,1.38,0.54,0.00
3,16,2014-01-22 02:17:45,RF05,"Convective outflow, 2 - North of Guam",AWAS,12916.369141,17.191528,150.000214,344.0,34.0,...,0.40,0.15,0.99,NaN,0.24,0.80,3.79,1.38,0.65,0.00
4,17,2014-01-22 02:29:00,RF05,"Convective outflow, 2 - North of Guam",AWAS,12914.645508,18.010277,149.249603,556.0,75.0,...,0.41,0.17,1.01,0.42,0.25,0.81,4.13,2.66,0.99,1.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,465,2014-02-24 23:36:58,RF15,"Jet crossing and LS survey, 4",AWAS,13228.544922,17.312660,144.976944,535.0,59.0,...,0.41,0.13,0.81,0.25,0.09,0.33,1.62,0.84,0.16,0.11
152,466,2014-02-24 23:51:51,RF15,"Jet crossing and LS survey, 4",AWAS,13213.697266,19.353573,145.088547,561.0,60.0,...,0.41,0.16,0.86,0.25,0.14,0.34,1.20,0.86,0.19,0.18
153,467,2014-02-25 00:06:52,RF15,"Jet crossing and LS survey, 4",AWAS,13181.125977,21.398539,145.188400,492.0,59.0,...,0.40,0.15,0.81,0.13,0.10,0.37,1.62,0.71,0.15,0.08
154,468,2014-02-25 00:21:51,RF15,"Jet crossing and LS survey, 4",AWAS,13135.843750,23.419264,145.146896,493.0,63.0,...,0.42,0.13,0.79,NaN,0.09,0.33,1.06,0.51,0.09,0.03


In [14]:
#### ------------------- GET TOGA SEGMENTS THAT CORRESPOND TO AWAS TIMES 
# make a new dataframe for toga with AWAS time, plus average of all tracers in the segment
# want one UT value per segment 
toga_segments = pd.DataFrame()
for col in toga_df.columns: 
    toga_segments[col] = ''
toga_segments = toga_segments.drop(columns = 'Notes')

# grab everything TOGA +/- 3 minutes from that awas sample 
for idx in awas_df_ut.iterrows():
    t = idx[1].Time_UTC
    # ----- get time of awas sample, +/- 3 minutes
    m = 3
    s = 60*m
    ts = t - datetime.timedelta(0,s)
    te = t + datetime.timedelta(0,s) 
    # ----- get toga segment that corresponds to that time 
    toga_seg = toga_df[(toga_df.Time_UTC > ts) & (toga_df.Time_UTC < te)]
    toga_seg_sum = toga_seg.mean()
    # ----- if there are toga meas in that time, append 
    if (len(toga_seg) > 0):
        # ----- fill in what was lost from mean 
        time = pd.Series({'Time_UTC': t})
        flight = pd.Series({'Flight': toga_seg['Flight'].iloc[0]})
        inst = pd.Series({'Intrument': 'TOGA'})
        toga_seg_toappend = pd.DataFrame(pd.concat([time, flight, inst, toga_seg_sum]))
        toga_seg_toappend = toga_seg_toappend.transpose()
        # add to dataframe! 
        toga_segments = toga_segments.append(toga_seg_toappend, ignore_index=True)
    else:
        toga_seg_filled = toga_seg
        toga_seg_filled = toga_seg_filled.drop(columns = 'Notes')
        # fill ins, qualitative 
        time = pd.Series({'Time_UTC': t})
        flight = idx[1].Flight
        ggalt = idx[1].GGALT
        gglat = idx[1].GGLAT
        gglon = idx[1].GGLON
        inst = pd.Series({'Intrument': 'TOGA'})
        # tracers make nan 
        toga_seg_filled = toga_seg_filled.append(pd.Series(), ignore_index=True)
        toga_seg_filled['Time_UTC'] = t
        toga_seg_filled['Flight'] = flight[0]
        toga_seg_filled['Intrument'] = inst[0]
        toga_seg_filled['GGALT'] = ggalt
        toga_seg_filled['GGLAT'] = gglat
        toga_seg_filled['GGLON'] = gglon
        # add to dataframe
        toga_segments = toga_segments.append(toga_seg_filled, ignore_index=True)


toga_segments[toga_segments['Flight'] == 'RF05']

/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:41: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Time_UTC,Flight,Intrument,GGALT,GGLAT,GGLON,Acetaldehyde,n_Butane,i_Butane,Benzene,C3H8,Methanol,Bromoform,Tetrachloroethylene,Dibromomethane,Dichloromethane,Chloroform,Carbon_Tetrachloride
0,2014-01-22 01:44:00,RF05,TOGA,12926.8,12.9629,149.367,2.5,0.796667,1.40333,3.64,29,39.6667,0.05,0.623333,0.924667,38.4433,7.85333,92.0333
1,2014-01-22 01:55:15,RF05,TOGA,12923.6,14.4463,149.818,2.5,1.08,0.843333,3.74667,32.3333,59.6667,0.134667,0.796667,0.981667,37.6733,8,90.8333
2,2014-01-22 02:06:30,RF05,TOGA,12922.4,15.7248,150.001,2.5,0.25,0.25,3.72667,26,120,0.316333,0.69,0.886333,37.2633,7.85667,90.3
3,2014-01-22 02:17:45,RF05,TOGA,12917.7,17.2771,150,2.5,1.35333,1.42667,3.47,30.3333,138,0.372667,0.583333,0.983333,34.7333,7.31333,90.5333
4,2014-01-22 02:29:00,RF05,TOGA,12912.6,18.0094,149.329,NaN,13.14,8.67,12.85,73,237,0.563333,1.14,0.999333,44.9667,8.58667,90.1667
5,2014-01-22 02:37:56,RF05,TOGA,13113,18.0209,148.043,2.5,3.96333,3.77333,7.11333,43,152.333,0.579,0.953333,1.12967,40.1767,8.14333,89.2
6,2014-01-22 02:42:44,RF05,TOGA,13216.8,18.0231,147.531,NaN,4.01333,3.38667,7.51333,35.6667,138.667,0.506,0.98,0.921333,41.35,8.28333,89.8
7,2014-01-22 02:53:59,RF05,TOGA,13206.8,18.0209,146.005,NaN,4.38333,2.81,8.49,32,188.667,0.55,1.1,0.864667,43.05,8.70667,90.9667
8,2014-01-22 03:33:52,RF05,TOGA,12583.3,18.5289,140.833,2.5,3.35333,2.7,6.38333,23,84,0.407,0.85,0.946,42.21,8.56667,91.9333
9,2014-01-22 03:48:51,RF05,TOGA,12268.7,18.4179,142.243,2.5,4.34,3.17,8.19333,32.3333,170.333,0.478667,1.05333,1.001,42.7067,8.37,91.9667


In [12]:
#### ------------------- pickle and save 
awas_segments = awas_df_ut
awas_segments.to_pickle("./awas_segments_noRF06.pkl")
toga_segments.to_pickle("./toga_segments_noRF06.pkl")